# Συλλογή δεδομένων

Επικεντρωθήκαμε στην κοινότητα "Φυτά, Βότανα Μυστικά" του Facebook όπου οι διαχειριστές κοινοποιούν άρθρα δημοσιευμένα σε σελίδες και blogs. Η συντριπτική πλειψηφία των αναρτήσεων προέρχεται από τη σελίδα ["Προϊόντα της Φύσης"](https://www.proionta-tis-fisis.com/). Επίσης, υπάρχουν λίγες αναρτήσεις από τις εξής πηγές [medialabnews.gr IATRIKANEA](https://medlabgr.blogspot.com/), [itrofi](https://www.itrofi.gr/), οι οποίες έχουν ειδικές κατηγορίες για βότανα και διατροφή αλλά και τις [Εναλλακτική Δράση](https://enallaktikidrasi.com/), [awekengr.com](https://www.awakengr.com/), [EarthShareMe.com](https://earthshareme.com/), [flowmag](https://www.flowmagazine.gr/), στις οποίες αναρτώνται καποια σχετικά άρθρα ανάμεσα σε άλλα. Σκεφτήκαμε να συλλέξουμε τα δεδομένα κατευθείαν από τις σελίδες/blogs.

Σε πρώτη φάση χρησιμοποιούμε μόνο τη σελίδα "Προϊόντα της Φύσης" και συγκεκριμένα την κατηγορία των βοτάνων.

In [1]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np

# get all links for articles in the category of votana (multiple pages) from proionta-tis-fysis
http = httplib2.Http()
useful_links = []
for counter in range(1,68):
    status, response = http.request('https://www.proionta-tis-fisis.com/category/votana/page/' + str(counter) + '/')
    #discard irrelevant links
    new_links = [link['href'] for link in BeautifulSoup(response, parse_only=SoupStrainer('a')).find_all('a', href=True) 
                    if ('category' not in link['href']) & ('#' not in link['href']) & ('epikoinonia' not in link['href'])
                    & ('shop' not in link['href']) & ('twitter' not in link['href']) & ('facebook' not in link['href']) 
                    & ('author' not in link['href']) & ('javascript' not in link['href']) & ('mailto' not in link['href'])
                    & ('oroi-xrisis-istotopou' not in link['href'])
                    & (link['href']!='https://www.proionta-tis-fisis.com/prosoxi')
                    & (link['href']!='https://www.proionta-tis-fisis.com/') 
                    & (link['href']!='https://www.proionta-tis-fisis.com')]
    useful_links += new_links
    
useful_links = np.unique(useful_links)

In [2]:
for link in useful_links:
    print(link)

https://www.proionta-tis-fisis.com/10-apo-tis-ekpliktikes-hriseis-tis-alois/
https://www.proionta-tis-fisis.com/10-hriseis-gia-tin-iperohi-levanta/
https://www.proionta-tis-fisis.com/10-logi-gia-na-fitepsis-rigani-sti-glastra/
https://www.proionta-tis-fisis.com/10-votana-pou-mporoun-na-nikoun-tis-flegmones/
https://www.proionta-tis-fisis.com/15-logoi-na-baleis-stin-zoi-sou-to-gaidouragkatho/
https://www.proionta-tis-fisis.com/20-fysika-antibiotika-thorakizoun-ton-organismo/
https://www.proionta-tis-fisis.com/25-therapeftikes-hriseis-tou-elaiou-cannabis-cbd/
https://www.proionta-tis-fisis.com/3000-votana-echoun-antikarkiniki-drasi-dite-pia/
https://www.proionta-tis-fisis.com/35-votana-simmaxoi-gia-kali-ygieia/
https://www.proionta-tis-fisis.com/4-votana-sta-opia-ipoklinete-i-epistimi/
https://www.proionta-tis-fisis.com/7-prosarmogona-votana-pou-voithoun-na-meiosoume-ta-epipeda-kortizolis/
https://www.proionta-tis-fisis.com/8-super-votana-pou-empistevonte-ke-i-giatri/
https://www.proiont

In [3]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

articles = []
for link in useful_links:
    article = url_to_string(link)
    articles.append(article)

In [4]:
len(articles)

673

In [5]:
len(useful_links)

673

In [7]:
clean_articles = []
for article in articles:
    article = article.split('ΠερισσοτεραΑναζήτησηΑρχική', 1)[1]
    article = article.split('Προηγούμενο άρθρο', 1)[0]
    clean_articles.append(article)

In [8]:
import pandas as pd

df1 = pd.DataFrame(list(zip(useful_links, clean_articles)), columns =['link', 'text']) 
df1.to_csv('../data/proionta_tis_fisis_votana.csv', index = False)
df1

,link,text
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...
...,...,...
668,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...
669,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...
670,https://www.proionta-tis-fisis.com/yparhei-zoi...,Τροφή για Σκέψη Υπάρχει Ζωή ΠΡΙΝ τον Θάνατο...
671,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of..."


In [27]:
import pandas as pd
df1 = pd.read_csv('../data/proionta_tis_fisis_votana.csv')

In [28]:
# discard some remaining irrelevant entries
df1 = df1.drop(df1.index[df1.index[df1['link'] == 'https://www.proionta-tis-fisis.com/i-zoi-einai-mikri-gia-na-zoume-kleismenoi-sto-asfales-koutaki-mas/'].tolist()
                + df1.index[df1['link'] == 'https://www.proionta-tis-fisis.com/synaisthimatiki-yperfagia-aisthima-katoterotitas-kai-antimetopisi/'].tolist()
                + df1.index[df1['link'] == 'https://www.proionta-tis-fisis.com/to-synoliko-oikologiko-apotypoma-ton-ananeosimon-pigon-energeias-den-ehei-dierevnithei-eparkos/'].tolist()
                + df1.index[df1['link'] == 'https://www.proionta-tis-fisis.com/tropoi-syntirisis-ton-trofimon-pro-psygeiou/'].tolist()
                + df1.index[df1['link'] == 'https://www.proionta-tis-fisis.com/yparhei-zoi-prin-ton-thanato/'].tolist()])
df1

,link,text
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...
...,...,...
667,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...
668,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...
669,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...
671,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of..."


In [29]:
df1.to_csv('../data/proionta_tis_fisis_votana.csv', index = False)

Σε αυτό το σημείο πλέον, όλες οι καταχωρήσεις περιέχουν το επιθυμητό κείμενο και όχι έξτρα πληροφορίες στην αρχή και στο τέλος. Επίσης, δεν υπάρχουν άρθρα με εντελώς άλλη θεματολογία. Κάποια είναι πιο σχετικά από άλλα για αυτό που θέλουμε να κάνουμε αλλά λογικά κάθε άρθρο θα περιέχει τουλάχιστον μία οντότητα.

Ακολουθούμε την ίδια διαδικασία για τη σελίδα "itrofi" τώρα, πάλι συγκεκριμένα για τα βότανα.

In [7]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import numpy as np

# get all links for articles in the category of votana (multiple pages) from itrofi
http = httplib2.Http()
useful_links = []
for counter in range(0,10):
    status, response = http.request('https://www.itrofi.gr/fytika/votana?page=' + str(counter))
    #discard irrelevant links
    new_links = ['https://www.itrofi.gr' + link['href'] for link in BeautifulSoup(response, parse_only=SoupStrainer('a'))
                 .find_all('a', href=True) if ('/fytika/votana/article/' in link['href'])]
    useful_links += new_links
    
useful_links = np.unique(useful_links)

In [8]:
for link in useful_links:
    print(link)

https://www.itrofi.gr/fytika/votana/article/1012/tsai-apo-hrysanthema-halaronei-kai-prostateyei
https://www.itrofi.gr/fytika/votana/article/1013/tsai-kai-adynatisma
https://www.itrofi.gr/fytika/votana/article/1035/ftelia-thaymatoyrgo-votano
https://www.itrofi.gr/fytika/votana/article/1038/10-votana-gia-na-kopsete-kapnisma
https://www.itrofi.gr/fytika/votana/article/1087/valeriana-ola-osa-prepei-na-gnorizeis
https://www.itrofi.gr/fytika/votana/article/1106/botana-kai-mpaharika-toy-erota
https://www.itrofi.gr/fytika/votana/article/1111/i-therapeytiki-dynami-ton-votanon
https://www.itrofi.gr/fytika/votana/article/1116/gaidoyragkatho-neo-antikarkiniko-farmako-gia-sykoti
https://www.itrofi.gr/fytika/votana/article/1117/lemonohorto-gia-prolipsi-alla-kai-gia-therapeia
https://www.itrofi.gr/fytika/votana/article/1165/gnoriste-ta-prosarmogona-ta-nea-votana-poy-katapolemoyn-stress
https://www.itrofi.gr/fytika/votana/article/1168/antimetopiste-ti-naytia-me-tsai-apo-votana
https://www.itrofi.gr/fy

In [9]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

# retrieve all articles from the links that we gathered before
articles = []
for link in useful_links:
    article = url_to_string(link)
    articles.append(article)

In [10]:
len(useful_links)

105

In [11]:
len(articles)

105

In [13]:
import pandas as pd

df2 = pd.DataFrame(list(zip(useful_links, articles)), columns =['link', 'text']) 
df2.to_csv('../data/itrofi_votana.csv', index=False)
df2

,link,text
0,https://www.itrofi.gr/fytika/votana/article/10...,"Τσάι από χρυσάνθεμα, χαλαρώνει και προστατεύε..."
1,https://www.itrofi.gr/fytika/votana/article/10...,Τσάι και αδυνάτισμα | iTrofi ...
2,https://www.itrofi.gr/fytika/votana/article/10...,Φτελιά: το θαυματουργό βότανο… | iTrofi ...
3,https://www.itrofi.gr/fytika/votana/article/10...,10 βότανα για να κόψετε το κάπνισμα | iTrofi ...
4,https://www.itrofi.gr/fytika/votana/article/10...,Βαλεριάνα: Όλα όσα πρέπει να γνωρίζεις… | iTr...
...,...,...
100,https://www.itrofi.gr/fytika/votana/article/96...,Τα αντικαρκινικά οφέλη της ρίγανης | iTrofi ...
101,https://www.itrofi.gr/fytika/votana/article/97...,"Αιθέριο έλαιο Ρίγανης: Θεραπεύει, ανακουφίζει..."
102,https://www.itrofi.gr/fytika/votana/article/98...,5 βότανα & μπαχαρικά που ελέγχουν την χοληστε...
103,https://www.itrofi.gr/fytika/votana/article/98...,Τσάι και αποτοξίνωση…. | iTrofi ...


Συγχωνεύουμε τα δύο προηγούμενα dtataframes, σε ένα dataset και τα αποθηκεύουμε σε ένα αρχείο, καθώς τα δεδομένα βρίσκονται στην ίδια ακριβώς μορφή.

In [16]:
df1 = pd.read_csv('../data/proionta_tis_fisis_votana.csv')
df2 = pd.read_csv('../data/itrofi_votana.csv')
df_texts = df1.append(df2, ignore_index=True)
df_texts

In [19]:
df_texts.to_csv('../data/votana_total.csv', index=False)